In [ ]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import os
import math

from scipy.optimize import curve_fit
from scipy.special import gamma

In [ ]:
def readRootTrees(runNumber: int) -> dict:
    """
    Reads all trees from a ROOT file and returns them as a dictionary.
    The keys are the tree names and the values are pandas DataFrames.

    Args:
        runNumber (int): The run number of the ROOT file.

    Returns:
        dict: A dictionary where keys are tree names (str) and
              values are pandas DataFrames. Returns an empty dictionary
              if the file cannot be opened or contains no trees.
    """

    dataFilePath = 'Data/'
    dataFile = f'sim.{runNumber}.root'
    fullFilePath = os.path.join(dataFilePath, dataFile)
    
    try:
        with uproot.open(fullFilePath) as rootFile:
            dataframes = {}
            for treeName in rootFile.keys():
                if isinstance(rootFile[treeName], uproot.behaviors.TTree.TTree):
                    tree = rootFile[treeName]
                    try:
                        df = tree.arrays(library='pd')
                        dataframes[treeName] = df
                    except Exception as e:
                        print(f"Error reading tree '{treeName}': {e}")
            return dataframes
    except Exception as e:
        print(f"Error opening or reading ROOT file '{fullFilePath}': {e}")
        return {}

In [ ]:
simData = readRootTrees(5000)
for key in simData:
    globals()[key[:-6]] = simData[key]
    print(key[:-6])

In [ ]:
print(metaData.iloc[0])

In [ ]:
def plotFieldLines(fieldLines, metaData, plotName) -> None:
    """
    Generates 3D and 2D plots visualizing electric field lines around a pixel.

    The plots display the field lines, the pixel geometry, and the hole
    in the mesh. The 2D plots provide projections of the 3D data onto
    different planes (xz, yz, and xy).

    Args:
        fieldLines: A Pandas DataFrame containing the xyz coordinates of the field lines.
        metaData (DataFrame): A DataFrame containing metadata about the detector geometry.
        plotName (str): A string used to title the plots.

    Returns:
        None
    """
    # Extract relevant geometric parameters from metadata.
    halfPitch = metaData['Pitch'].iloc[0]/2.
    halfPixel = metaData['Pixel Width'].iloc[0]/2.
    halfMeshThickness = metaData['Mesh Thickness'].iloc[0]/2.
    pixelHeight = -halfMeshThickness - metaData['Mesh Standoff'].iloc[0]
    cathodeHeight = halfMeshThickness + metaData['Cathode Height'].iloc[0]
    holeRadius = metaData['Hole Radius'].iloc[0]

    # Define the vertices of the pixel outline.
    pixelX = [-halfPixel, -halfPixel, halfPixel, halfPixel, -halfPixel]
    pixelY = [-halfPixel, halfPixel, halfPixel, -halfPixel, -halfPixel]
    pixelZ = [pixelHeight] * 5

    # Define a circle representing the hole in the mesh.
    hole = plt.Circle((0, 0), holeRadius, facecolor='none', edgecolor='k', ls=':', lw=2)
    holeInPlane = [-holeRadius, holeRadius, holeRadius, -holeRadius, -holeRadius]
    holeZ = [-halfMeshThickness, -halfMeshThickness, halfMeshThickness, halfMeshThickness, -halfMeshThickness]

    # Group field lines together based on the line ID
    groupedFieldLines = fieldLines.groupby('Field Line ID')

    # Create the 3D figure and subplot.
    fig3D = plt.figure(figsize=(16, 8))
    fig3D.suptitle(f'Field Lines - {plotName}')
    ax3D = fig3D.add_subplot(111, projection='3d')

    # Create the 2D figure and subplots for different projections.
    fig2D = plt.figure(figsize=(14, 7))
    fig2D.suptitle(f'Field Lines - {plotName}')
    ax11 = fig2D.add_subplot(221)
    ax12 = fig2D.add_subplot(223)
    ax13 = fig2D.add_subplot(122)

    # Iterate through each field line and plot it in both 3D and 2D subplots.
    for lineID, fieldLine in groupedFieldLines:
        ax3D.plot(fieldLine['Field Line x'],
                  fieldLine['Field Line y'],
                  fieldLine['Field Line z'])

        ax11.plot(fieldLine['Field Line x'],
                  fieldLine['Field Line z'])
        ax12.plot(fieldLine['Field Line y'],
                  fieldLine['Field Line z'])
        ax13.plot(fieldLine['Field Line x'],
                  fieldLine['Field Line y'])

    # Plot the pixel outline
    ax3D.plot(pixelX, pixelY, pixelZ, label='Pixel', c='m')
    ax11.plot(pixelX, pixelZ, label='Pixel', c='m')
    ax12.plot(pixelY, pixelZ, label='Pixel', c='m')
    ax13.plot(pixelX, pixelY, label='Pixel', c='m')

    # Plot the hole
    ax11.plot(holeInPlane, holeZ, label='Hole', c='k', ls=':')
    ax12.plot(holeInPlane, holeZ, label='Hole', c='k', ls=':')
    ax13.add_patch(hole)

    # Set axis labels
    ax3D.set_xlabel('x')
    ax3D.set_ylabel('y')
    ax3D.set_zlabel('z')

    ax11.set_xlabel('x')
    ax11.set_ylabel('z')
    ax12.set_xlabel('y')
    ax12.set_ylabel('z')
    ax13.set_xlabel('x')
    ax13.set_ylabel('y')

    # Set axis limits
    ax3D.set_xlim(-halfPitch, halfPitch)
    ax3D.set_ylim(-halfPitch, halfPitch)
    ax3D.set_zlim(pixelHeight, cathodeHeight)

    ax11.set_xlim(-halfPitch, halfPitch)
    ax11.set_ylim(pixelHeight, cathodeHeight)
    ax12.set_xlim(-halfPitch, halfPitch)
    ax12.set_ylim(pixelHeight, cathodeHeight)
    ax13.set_xlim(-halfPitch, halfPitch)
    ax13.set_ylim(-halfPitch, halfPitch)

    # Add grid lines
    ax11.grid()
    ax12.grid()
    ax13.grid()

    return

In [ ]:
fieldLines = fieldLineData
plotFieldLines(fieldLines, metaData, 'Cathode')

aboveMeshLines = meshFieldLineData[meshFieldLineData['Mesh Line Location']==1]
plotFieldLines(aboveMeshLines, metaData, 'Above Mesh')

belowMeshLines = meshFieldLineData[meshFieldLineData['Mesh Line Location']==-1]
plotFieldLines(belowMeshLines, metaData, 'Below Mesh')

In [ ]:
def plotLocations(particleData, metaData, plotName) -> None:
    """
    Generates 2D histograms to visualize the inital and final locations
    of simulated particles from many avalanches.

    The pixel geometry and the hole in the mesh are included.

    Args:
        particleData (DataFrame): A Pandas DataFrame containing the initial and final
                                  coordinates for a type of particle.
        metaData (DataFrame): A DataFrame containing metadata about the detector geometry.
        plotName (str): A string used to title the plots.

    Returns:
        None
    """
    # Extract relevant geometric parameters from metadata.
    halfPitch = metaData['Pitch'].iloc[0]/2.
    halfPixel = metaData['Pixel Width'].iloc[0]/2.
    halfMeshThickness = metaData['Mesh Thickness'].iloc[0]/2.
    pixelHeight = -halfMeshThickness - metaData['Mesh Standoff'].iloc[0]
    cathodeHeight = halfMeshThickness + metaData['Cathode Height'].iloc[0]
    holeRadius = metaData['Hole Radius'].iloc[0]

    # Define the vertices of the pixel outline.
    pixelX = [-halfPixel, -halfPixel, halfPixel, halfPixel, -halfPixel]
    pixelY = [-halfPixel, halfPixel, halfPixel, -halfPixel, -halfPixel]
    pixelZ = [pixelHeight] * 5

    # Define a circle representing the hole in the mesh.
    hole = plt.Circle((0, 0), holeRadius, facecolor='none', edgecolor='r', ls=':', lw=2)
    hole2 = plt.Circle((0, 0), holeRadius, facecolor='none', edgecolor='r', ls=':', lw=2)
    holeInPlane = [-holeRadius, holeRadius, holeRadius, -holeRadius, -holeRadius]
    holeZ = [-halfMeshThickness, -halfMeshThickness, halfMeshThickness, halfMeshThickness, -halfMeshThickness]

    numBins = 51

    # Create the figure and add subplots
    fig = plt.figure(figsize=(12, 12))
    fig.suptitle(f'Particle Locations: {plotName}')
    
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    # Plot data
    ax1.hist2d(particleData['Initial x'], \
               particleData['Initial y'], \
               bins=numBins, range=[[-halfPitch, halfPitch], [-halfPitch, halfPitch]])
    ax2.hist2d(particleData['Final x'], \
               particleData['Final y'], \
               bins=numBins, range=[[-halfPitch, halfPitch], [-halfPitch, halfPitch]])
    ax3.hist2d(particleData['Initial x'], \
               particleData['Initial z'], \
               bins=numBins, range=[[-halfPitch, halfPitch], [pixelHeight, cathodeHeight]])
    ax4.hist2d(particleData['Final x'], \
               particleData['Final z'], \
               bins=numBins, range=[[-halfPitch, halfPitch], [pixelHeight, cathodeHeight]])

    # Plot pixel outline
    ax1.plot(pixelX, pixelY, label='Pixel', c='m')
    ax2.plot(pixelX, pixelY, label='Pixel', c='m')
    ax3.plot(pixelX, pixelZ, label='Pixel', c='m')
    ax4.plot(pixelX, pixelZ, label='Pixel', c='m')

    # Plot the hole
    ax1.add_patch(hole)
    ax2.add_patch(hole2)
    ax3.plot(holeInPlane, holeZ, label='Hole', c='r', ls=':')
    ax4.plot(holeInPlane, holeZ, label='Hole', c='r', ls=':')

    # Set titles
    ax1.set_title('Initial')
    ax2.set_title('Final')
    ax3.set_title('Initial')
    ax4.set_title('Final')
    
    # Set axis labels
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax3.set_xlabel('x')
    ax3.set_ylabel('z')
    ax4.set_xlabel('x')
    ax4.set_ylabel('z')

    return

In [ ]:
newElectrons = electronData[electronData['Electron ID'] != 0]
plotLocations(newElectrons, metaData, 'Electrons')

positiveIons = ionData[(ionData['Ion Charge'] == 1) & (ionData['Electron ID'] != 0)]
plotLocations(positiveIons, metaData, 'Positive Ions')

negativeIons = ionData[(ionData['Ion Charge'] == -1)]
plotLocations(negativeIons, metaData, 'Negative Ions')

In [ ]:
def plotSingleAvalanche(electronTrackData, avalancheID, metaData, plotName) -> None:
    """
    Plots the drift lines of all electrons from a specified simulated electron avalanche.

    The pixel geometry and the hole in the mesh are included.

    Args:
        electronTrackData (DataFrame): A Pandas DataFrame containing the xyz coordinates of electron tracks.
        avalancheID (int): The ID number of the desired avalanche to plot
        metaData (DataFrame): A DataFrame containing metadata about the detector geometry.
        plotName (str): A string used to title the plots.

    Returns:
        None
    """
    # Extract relevant geometric parameters from metadata.
    halfPitch = metaData['Pitch'].iloc[0]/2.
    halfPixel = metaData['Pixel Width'].iloc[0]/2.
    halfMeshThickness = metaData['Mesh Thickness'].iloc[0]/2.
    pixelHeight = -halfMeshThickness - metaData['Mesh Standoff'].iloc[0]
    cathodeHeight = halfMeshThickness + metaData['Cathode Height'].iloc[0]
    holeRadius = metaData['Hole Radius'].iloc[0]

    # Define the vertices of the pixel outline.
    pixelX = [-halfPixel, -halfPixel, halfPixel, halfPixel, -halfPixel]
    pixelY = [-halfPixel, halfPixel, halfPixel, -halfPixel, -halfPixel]
    pixelZ = [pixelHeight] * 5

    # Define a circle representing the hole in the mesh.
    hole = plt.Circle((0, 0), holeRadius, facecolor='none', edgecolor='k', ls=':', lw=2)
    holeInPlane = [-holeRadius, holeRadius, holeRadius, -holeRadius, -holeRadius]
    holeZ = [-halfMeshThickness, -halfMeshThickness, halfMeshThickness, halfMeshThickness, -halfMeshThickness]


    # Isolate the desired avalanche
    singleAvalancheData = electronTrackData[electronTrackData['Avalanche ID'] == avalancheID]
    
    # Group drift lines together based on the electron ID
    groupedAvalancheData = singleAvalancheData.groupby('Electron ID')

    # Create the figure and add subplots
    fig = plt.figure(figsize=(14, 7))
    fig.suptitle(f'Avalanche: {plotName} (ID #{groupedAvalancheData['Avalanche ID'].min().iloc[0]})')
    
    ax11 = fig.add_subplot(221)
    ax12 = fig.add_subplot(223)
    ax13 = fig.add_subplot(122)

    # Iterate through the drift lines of each electron
    for electronID, driftLine in groupedAvalancheData:
        ax11.plot(driftLine['Drift x'], \
                  driftLine['Drift z'], \
                  linewidth=.5)
        ax12.plot(driftLine['Drift y'], \
                  driftLine['Drift z'], \
                  linewidth=.5)
        ax13.plot(driftLine['Drift x'], \
                  driftLine['Drift y'], \
                  linewidth=.5)


    # Plot pixel outline
    ax11.plot(pixelX, pixelZ, label='Pixel', c='m')
    ax12.plot(pixelY, pixelZ, label='Pixel', c='m')
    ax13.plot(pixelX, pixelY, label='Pixel', c='m')

    # Plot the hole
    ax11.plot(holeInPlane, holeZ, label='Hole', c='k', ls=':')
    ax12.plot(holeInPlane, holeZ, label='Hole', c='k', ls=':')
    ax13.add_patch(hole)

    # Axis labels
    ax11.set_xlabel('x')
    ax11.set_ylabel('z')
    ax12.set_xlabel('y')
    ax12.set_ylabel('z')
    ax13.set_xlabel('x')
    ax13.set_ylabel('y')

    #Axis limits
    ax11.set_xlim(-halfPitch, halfPitch)
    ax11.set_ylim(pixelHeight, cathodeHeight)
    ax12.set_xlim(-halfPitch, halfPitch)
    ax12.set_ylim(pixelHeight, cathodeHeight)
    ax13.set_xlim(-halfPitch, halfPitch)
    ax13.set_ylim(-halfPitch, halfPitch)

    # Add grid lines
    ax11.grid()
    ax12.grid()
    ax13.grid()

    return

In [ ]:
averageAvalancheID = electronData[electronData['Electron ID']==math.floor(electronData['Electron ID'].mean())]['Avalanche ID'].iloc[0]
plotSingleAvalanche(electronTrackData, averageAvalancheID, metaData, 'Average')

largestAvalancheID = electronData[electronData['Electron ID']==electronData['Electron ID'].max()]['Avalanche ID'].iloc[0]
plotSingleAvalanche(electronTrackData, largestAvalancheID, metaData, 'Largest')

In [ ]:
def plotAvalancheSize(avalancheData, metaData, binWidth=1) -> None:
    """
    """
    runNo = metaData['runNo'].iloc[0]
    gain = avalancheData['Total Electrons'].mean()

    bins = np.arange(avalancheData['Total Electrons'].min(), \
                     avalancheData['Total Electrons'].max()+1, \
                     binWidth)
    
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(f'Avalanche Size: Run {runNo}')
    
    ax = fig.add_subplot(111)
    
    ax.hist(avalancheData['Total Electrons'], \
             bins=bins, label='Simulation')
    
    ax.axvline(x=gain, \
               c='g', ls='--', label=f'Simulated Gain = {gain:.0f}')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('')
    
    ax.legend()
    ax.grid()
    
    return
    

In [ ]:
plotAvalancheSize(avalancheData, metaData)
test = avalancheData[avalancheData['Total Electrons'] > 1]
plotAvalancheSize(test, metaData)

In [ ]:
def myPolya(n, nBar, theta):
    return (1/nBar)*np.power(theta+1, theta+1)/gamma(theta+1)*np.power(n/nBar, theta)*np.exp(-(n/nBar)*(theta+1))

def myPolya2(n, nBar, theta):
    theta2 = theta-1
    return myPolya(n, nBar, theta2)
    
def fitAvalanchePolya(avalancheData, metaData, binWidth=1):
    """
    """
    runNo = metaData['runNo'].iloc[0]
    gain = avalancheData['Total Electrons'].mean()

    bins = np.arange(avalancheData['Total Electrons'].min(), \
                     avalancheData['Total Electrons'].max()+1, \
                     binWidth)
    binCenters = bins[:-1] + binWidth/2.

    counts, _ = np.histogram(avalancheData['Total Electrons'], \
                                  bins=bins)
    prob = counts/len(avalancheData['Total Electrons'])/binWidth

    # Get error
    countErr = np.where(counts == 0, 1, np.sqrt(counts))
    probErr = countErr/len(avalancheData['Total Electrons'])/binWidth

    # Set initial fitting parameters with some bounds
    initial = [gain, 1.]
    bounds = ([1, 0], [200, 10])

    popt, pcov = curve_fit(myPolya, binCenters, prob, p0=initial, bounds=bounds)
    perr = np.sqrt(np.diag(pcov))
    fitPolya = myPolya(binCenters, *popt)

    popt2, pcov2 = curve_fit(myPolya2, binCenters, prob, p0=initial, bounds=bounds)
    perr2 = np.sqrt(np.diag(pcov2))
    fitPolya2 = myPolya2(binCenters, *popt2)

    poptErr, pcovErr = curve_fit(myPolya, binCenters, prob, p0=initial, bounds=bounds, sigma=probErr)
    perrErr = np.sqrt(np.diag(pcovErr))
    fitPolyaErr = myPolya(binCenters, *poptErr)
    
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(f'Avalanche Size: Run {runNo}')
    
    ax = fig.add_subplot(111)
    
    ax.bar(bins[:-1], prob, \
           width=binWidth, align='edge', label='Simulated Data', color='b')
    ax.axvline(x=gain, \
               c='g', ls='--', label=f'Simulated Gain = {gain:.0f}e')

    ax.plot(binCenters, fitPolya, \
            'm-', lw=2, label=r'Fitted Polya ($\theta$' + f' = {popt[1]:.4f})')
    ax.axvline(x=popt[0], \
               c='m', ls=':', label=f'Fitted Gain = {popt[0]:.0f}e')

    ax.plot(binCenters, fitPolya2, \
            'c--', lw=2, label=r'Fitted Polya2 ($\theta$' + f' = {popt2[1]-1:.4f})')
    ax.axvline(x=popt2[0], \
               c='c', ls=':', label=f'Fitted Gain = {popt2[0]:.0f}e')

    ax.plot(binCenters, fitPolyaErr, \
            'r--', lw=2, label=r'Fitted Polya with Errors ($\theta$' + f' = {poptErr[1]:.4f})')
    ax.axvline(x=poptErr[0], \
               c='r', ls=':', label=f'Fitted Gain = {poptErr[0]:.0f}e')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('Probability')
    
    ax.legend()
    ax.grid()

    return

In [ ]:
fitAvalanchePolya(avalancheData, metaData, binWidth=1)